In [38]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Dropout
from keras import regularizers

In [2]:
medium_data = pd.read_csv('/Users/marcgrayson/Downloads/medium_data.csv')
medium_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6508 entries, 0 to 6507
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            6508 non-null   int64 
 1   url           6508 non-null   object
 2   title         6508 non-null   object
 3   subtitle      3479 non-null   object
 4   image         6361 non-null   object
 5   claps         6508 non-null   int64 
 6   responses     6508 non-null   object
 7   reading_time  6508 non-null   int64 
 8   publication   6508 non-null   object
 9   date          6508 non-null   object
dtypes: int64(3), object(7)
memory usage: 508.6+ KB


In [3]:
for i in range(10):
    rand_index = random.randint(0, len(medium_data) - 1)
    rand_title = medium_data.iloc[rand_index, 2]
    print(rand_title)

A 3-step guide to surviving guilt
30 Days With No Social Media, or News. A Personal Experiment
The Coming Cloud Gaming Wars
Correlation does not imply causation
How to be a Successful Entrepreneur, in Two Easy Steps
Programmers health
We Need to Talk About KFC’s Twitter Game
The 5 Classification Evaluation metrics every Data Scientist must know
Clustering Evaluation strategies
Why do kids take their own lives?


In [4]:
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace(u'\xa0',u' '))
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace('\u200a',' '))
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace(u'</strong>',u' '))
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace(u'<strong class="markup--strong markup--h3-strong">',u' '))


In [5]:
# Se crea una instancia del Tokenizer con un token especial "<oov>" para palabras fuera del vocabulario
tokenizer = Tokenizer(oov_token='<oov>')

# Se ajusta el tokenizador utilizando los datos de la columna 'title' del DataFrame medium_data
tokenizer.fit_on_texts(medium_data['title'])

# Se calcula el número total de palabras en el vocabulario, agregando 1 para el token "<oov>"
total_words = len(tokenizer.word_index) + 1

In [6]:
print("Total number of words: ", total_words-1)

Total number of words:  8237


In [7]:
input_sequences = []
for line in medium_data['title']:
    token_list = tokenizer.texts_to_sequences([line])[0]

    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [8]:
print("Total input sequences: ", len(input_sequences))

Total input sequences:  45749


In [9]:
print(input_sequences[5])

[4, 675, 64, 2, 451, 1518, 12]


In [10]:
medium_data.iloc[5,2]

'An Easy Introduction to SQL for Data Scientists'

In [11]:
# Encontrar la longitud máxima de las secuencias de entrada existentes
max_sequence_len = max([len(x) for x in input_sequences])

# Hacer que todas las secuencias de entrada tengan la misma longitud usando el relleno (padding)
# Se utiliza 'pre' para rellenar al inicio de cada secuencia
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Acceder a la segunda secuencia de entrada después del relleno
input_sequences[5]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          4,  675,   64,    2,  451, 1518,   12], dtype=int32)

In [12]:
# create features and label
x, labels = input_sequences[:,:-1],input_sequences[:,-1]
y = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [13]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(x, y, epochs=10, verbose=1)
#print model.summary() 
print(model)

Epoch 1/10


2023-07-02 17:58:36.714810: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1430/1430 [==============================] - 52s 35ms/step - loss: 7.2637 - accuracy: 0.0542
Epoch 2/10
1430/1430 [==============================] - 50s 35ms/step - loss: 6.5137 - accuracy: 0.0917
Epoch 3/10
1430/1430 [==============================] - 51s 36ms/step - loss: 5.9793 - accuracy: 0.1256
Epoch 4/10
1430/1430 [==============================] - 50s 35ms/step - loss: 5.4952 - accuracy: 0.1514
Epoch 5/10
1430/1430 [==============================] - 50s 35ms/step - loss: 5.0256 - accuracy: 0.1754
Epoch 6/10
1430/1430 [==============================] - 50s 35ms/step - loss: 4.5787 - accuracy: 0.2051
Epoch 7/10
1430/1430 [==============================] - 54s 38ms/step - loss: 4.1634 - accuracy: 0.2456
Epoch 8/10
1430/1430 [==============================] - 52s 36ms/step - loss: 3.7686 - accuracy: 0.2980
Epoch 9/10
1430/1430 [==============================] - 51s 36ms/step - loss: 3.4030 - accuracy: 0.3512
Epoch 10/10
1430/1430 [==============================] - 50s 35ms/step - lo

In [66]:
seed_text = "An Easy Introduction to SQL"
next_words = 3
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)
    predicted = np.argmax(predicted_probs, axis=-1)  # toma la clase con la probabilidad más alta
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)


An Easy Introduction to SQL for data scientists


In [56]:
model3 = Sequential()
model3.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model3.add(LSTM(100))  # Usamos 20 unidades en la capa LSTM
model3.add(Dense(total_words, activation='softmax'))  # Usamos tantas neuronas como palabras en el vocabulario y activación softmax
model3.compile(loss='categorical_crossentropy', optimizer='adam')
model3.fit(x, y, epochs=10, verbose=1)


Epoch 1/10
1430/1430 [==============================] - 30s 21ms/step - loss: 7.3430
Epoch 2/10
1430/1430 [==============================] - 31s 22ms/step - loss: 6.7378
Epoch 3/10
1430/1430 [==============================] - 30s 21ms/step - loss: 6.2850
Epoch 4/10
1430/1430 [==============================] - 30s 21ms/step - loss: 5.8886
Epoch 5/10
1430/1430 [==============================] - 30s 21ms/step - loss: 5.5174
Epoch 6/10
1430/1430 [==============================] - 30s 21ms/step - loss: 5.1651
Epoch 7/10
1430/1430 [==============================] - 30s 21ms/step - loss: 4.8320
Epoch 8/10
1430/1430 [==============================] - 29s 21ms/step - loss: 4.5123
Epoch 9/10
1430/1430 [==============================] - 30s 21ms/step - loss: 4.2057
Epoch 10/10
1430/1430 [==============================] - 30s 21ms/step - loss: 3.9094


In [57]:
model4 = Sequential()
model4.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model4.add(LSTM(20))  # Corrección aquí
model4.add(Dense(1)) 
model4.compile(loss='mape', optimizer='adam')
model4.fit(x, y, epochs=10, verbose=1)


Epoch 1/10
1430/1430 [==============================] - 13s 9ms/step - loss: 1201082.2500
Epoch 2/10
1430/1430 [==============================] - 12s 9ms/step - loss: 274872.5000
Epoch 3/10
1430/1430 [==============================] - 13s 9ms/step - loss: 197111.5469
Epoch 4/10
1430/1430 [==============================] - 12s 9ms/step - loss: 158638.2344
Epoch 5/10
1430/1430 [==============================] - 12s 9ms/step - loss: 120151.7031
Epoch 6/10
1430/1430 [==============================] - 12s 8ms/step - loss: 90930.0078
Epoch 7/10
1430/1430 [==============================] - 12s 9ms/step - loss: 73917.7734
Epoch 8/10
1430/1430 [==============================] - 13s 9ms/step - loss: 66569.0547
Epoch 9/10
1430/1430 [==============================] - 12s 9ms/step - loss: 63404.8086
Epoch 10/10
1430/1430 [==============================] - 12s 9ms/step - loss: 74548.3672


In [58]:
model5 = Sequential()
model5.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model5.add(Bidirectional(LSTM(150)))
model5.add(Dense(total_words, activation='softmax'))
model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x, y, epochs=10, verbose=1)
#print model.summary() 
print(model5)

Epoch 1/10
1430/1430 [==============================] - 49s 34ms/step - loss: 2.7699 - accuracy: 0.4589
Epoch 2/10
1430/1430 [==============================] - 50s 35ms/step - loss: 2.5003 - accuracy: 0.5076
Epoch 3/10
1430/1430 [==============================] - 48s 33ms/step - loss: 2.2630 - accuracy: 0.5502
Epoch 4/10
1430/1430 [==============================] - 48s 34ms/step - loss: 2.0509 - accuracy: 0.5907
Epoch 5/10
1430/1430 [==============================] - 49s 34ms/step - loss: 1.8621 - accuracy: 0.6262
Epoch 6/10
1430/1430 [==============================] - 49s 34ms/step - loss: 1.6924 - accuracy: 0.6629
Epoch 7/10
1430/1430 [==============================] - 49s 34ms/step - loss: 1.5446 - accuracy: 0.6895
Epoch 8/10
1430/1430 [==============================] - 49s 34ms/step - loss: 1.4158 - accuracy: 0.7153
Epoch 9/10
1430/1430 [==============================] - 49s 34ms/step - loss: 1.3017 - accuracy: 0.7391
Epoch 10/10
1430/1430 [==============================] - 49s 34m

In [64]:
seed_text = "An Easy Introduction to SQL"
next_words = 3
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model5.predict(token_list, verbose=0)
    predicted = np.argmax(predicted_probs, axis=-1)  # toma la clase con la probabilidad más alta
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)


An Easy Introduction to SQL unlimited jerry unlimited
